In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.fftpack as fft
from astropy.io import fits
from scipy import optimize
from scipy import signal
import poppy
import gpipsfs
import os
import pytz
import time
import re

In [ ]:
from matplotlib import rc
from matplotlib import rcParams
rc('xtick', labelsize=15) 
rc('ytick', labelsize=15)

rc('axes', labelsize=20) 
rc('axes', linewidth=1)

rcParams['axes.titlesize'] = 15
rcParams['legend.fontsize'] = 15
rcParams['patch.linewidth'] = 1
#rcParams['axes.titlepad'] = 30
rcParams['axes.labelpad'] = 7

# Telescope Dimensions

In [ ]:
outD = 7.77010            # primary diameter (m)
inD = 1.024               # inner M2 diameter (m)
n = 48                    # number sample points across the screen (Not the number of subapertures)
nacross = 43              # number of subapertures across the aperture
pscale = outD/(nacross)   # pixel size (m) of samples in pupil plane

# Define Aperture

In [ ]:
#  Aperture containing zeros
x = np.linspace(-(n)/2,(n)/2,n)*pscale 
y = np.linspace(-(n)/2,(n)/2,n)*pscale
mg = np.meshgrid(x,y)
ar = np.sqrt(np.sum((m**2 for m in mg)))
ap_outer = (ar <= (7.3)/2) #mask is slightly oversized because GPI does not correct the boundaries well
ap_inner = (ar <= 1.5/2)   

#ap_outer = (ar <= outD/2)
#ap_inner = (ar <= inD/2)   
ap = (ap_outer ^ ap_inner).astype(np.float)

#  Aperture containing nans
ap_nan = np.copy(ap.astype(np.float))  
ap_nan[np.where(ap==0)] = np.nan

# Make the spatial freq grid

Will use to plot against spatial PSDs

In [ ]:
kx = fft.fftshift(fft.fftfreq(n,pscale))
ky = fft.fftshift(fft.fftfreq(n,pscale))
mg = np.meshgrid(kx,ky)
kr = np.sqrt(np.sum((m**2 for m in mg))) 

# Phase Sample

1. name_list - list containing telemetry date strings
2. fname_list - list containing path to telemetry files

In [ ]:
def path_manager(desktop_name):
    
    if desktop_name == 'gpi_cruncher':
        rootdir = '/home/sda/mtallis/PhaseScripts/aotelem/Reduced/'
        save_path = '/home/sda/mtallis/Results/c_Eri/'
        samples_path = '/home/sda/mtallis/samples/c_Eri_samples.txt'
    
    if desktop_name == 'kipac':
        rootdir = '/Users/MelisaT/Documents/Research/GPIDomeSeeing/data/Results/c_Eri/'
        save_path = '/Users/MelisaT/Documents/Research/GPIDomeSeeing/data/Results/journal_figures/'
        samples_path = '/Users/MelisaT/Documents/Research/GPIDomeSeeing/data/datatables/c_Eri_samples.txt'
        
    if desktop_name == 'laptop':
        rootdir = '/Users/melisatallis/Documents/Research/GPIDomeSeeing/data/Results/c_Eri/'
        save_path = '/Users/melisatallis/Documents/Research/GPIDomeSeeing/data/Results/c_Eri/'
        samples_path = '/Users/melisatallis/Documents/Research/GPIDomeSeeing/data/Results/c_Eri/c_Eri_samples.txt'

    dstr = time.strftime('%Y%m%d')
    return rootdir,save_path, samples_path, dstr

In [ ]:
rootdir, save_path, samples_path, dstr = path_manager('gpi_cruncher')

In [ ]:
with open(samples_path,'r') as f:
    sample = f.read().splitlines() #outputs as a list of strings

Only use with gpi cruncher when images need to be processed

In [ ]:
fname_list = list()
name_list = list()

for i in sample:
    for root, dirs, files in os.walk(rootdir):
        for name in files:
            (base,ext) = os.path.splitext(name)
            if (ext in ('.fits')) and (i in base):
                full_name = os.path.join(root,name)
                #print(full_name)
                fname_list.append(full_name)  
                name_list.append(base[11:-12])

# Analysis Functions

1. processs_phase - Imports .fits & removes static aberrations in each phase  
2. radialProfile - Computes mean inside each radial bin starting from center
3. sp_power_spec - Computes time average of 2d DFT^2  
4. temp_power_spec - Computes actuator timeseries DFT^2 and averages over aperture
5. linear fit - fits a power law to psd. Behaves like a line in loglog space

In [ ]:
def import_fits(filepath):

    hdulist = fits.open(filepath,memmap=True)
    phase = hdulist[0].data.astype('float')

    return phase

In [ ]:
def remove_zernikes(phase):
    
    m1 = gpipsfs.GeminiPrimary().sample(npix=48)
    avg_phase = np.mean(phase*m1,axis=0)
    
    z_basis = poppy.zernike.zernike_basis_faster(nterms= 6, npix = 48)
    z_coeff = poppy.zernike.opd_expand_nonorthonormal(avg_phase, aperture=m1, nterms=6)
    thin_lens = np.sum(z_coeff[:,None,None]*z_basis[:,:,:],axis=0)
    c_phase = (phase - thin_lens[None,:,:])*m1
    c_phase[np.isnan(c_phase)]=0.

    return c_phase

In [ ]:
def convert_to_modal_basis(phase):
    
    timesteps, phx, phy = phase.shape 
    ap = gpipsfs.GPI_Apodizer().sample(npix = phx) # windowing function to smooth-out hard edges of aperture
    
    phFT = np.zeros((timesteps,phx,phy), dtype=complex)
    for t in np.arange(timesteps):
        phFT[t,:,:] = np.fft.fftshift(fft.fft2(phase[t,:,:]*ap)/np.sqrt(ap.sum()))
    print('Done with FT')

    # remove static aberrations from the signal 
    mft = np.mean(phFT,axis = 0) 
    phFT = phFT - mft[None,:,:]  
    
    return phFT

In [ ]:
def sp_power_spec(phFT):    
    
    timesteps, phx, phy = phFT.shape
    
    # compute 2d psd cube
    psd2D = np.zeros((timesteps, phx, phy),dtype=float)
    for k in np.arange(phx):
        for l in np.arange(phy):
            psd2D[:,k,l] = np.square(np.abs(phFT[:,k,l]))
    
    varpsd = np.sum(psd2D, axis=0)
    print('Done with PSD')    
    
    # compute radial average of 2d psd cube and frequency
    psd1D =  radialProfile(varpsd)
    
    return psd1D

In [ ]:
def process_phase(filepath):

    hdulist = fits.open(filepath,memmap=True)
    phase = hdulist[0].data.astype('float')
    avg_phase = np.nanmean(phase*ap_nan,axis=0)  # used to find average zernikes 

    # remove zernikes form cube
    z_basis = zernike.zernike_basis_faster(nterms= 6, npix = 48)
    z_coeff = zernike.opd_expand_nonorthonormal(avg_phase, aperture=ap, nterms=6)
    thin_lens = np.sum(z_coeff[:,None,None]*z_basis[:,:,:],axis=0)

    c_phase = (phase - thin_lens[None,:,:])*ap_nan
    c_phase[np.isnan(c_phase)]=0.
    
    return c_phase

In [ ]:
def radialProfile(image, center=None):
    """
    Calculate the avearge radial profile.

    image - The 2D image
    center - The [x,y] pixel coordinates used as the center. The default is 
             None, which then uses the center of the image (including 
             fracitonal pixels).
    
    """
    ## Calculate the indices from the image
    y,x = np.indices((image.shape)) # first determine radii of all pixels
    
    if not center:
        center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0])
     
    r = np.hypot(x - center[0], y - center[1]).astype(np.int) 
    
    n = np.bincount(r.ravel())
    sy = np.bincount(r.ravel(), image.ravel())
    mean = sy/n
    
    return mean

# old stuff

In [ ]:
def sp_power_spec(phase):    
    
    timesteps, phx, phy = phase.shape 
    phFT = np.zeros((timesteps,phx,phy), dtype=complex)
    for t in np.arange(timesteps):
        phFT[t,:,:] = fft.fftshift(fft.fft2(phase[t,:,:]))*2.0/ap.sum()
    print('Done with FT')
    
    # compute 2d psd cube
    psd2D = np.zeros((timesteps, phx, phy),dtype=float)
    for k in np.arange(phx):
        for l in np.arange(phy):
            psd2D[:,k,l] = np.square(np.abs(phFT[:,k,l]))
    
    avg_psd2D = np.mean(psd2D, axis=0)
    print('Done with PSD')    
    
    # compute radial average of 2d psd cube and frequency
    avg_psd1D =  radialProfile(avg_psd2D)
    
    return avg_psd1D

In [ ]:
def temp_power_spec(Y,sample_spacing):
    
    n = len(Y)
    dT = sample_spacing

    w = signal.blackman(n)
    P = np.fft.rfft(Y*w)
    norm = 2.0/n
    P = P * norm

    P2 = np.square(np.abs(P))
    #k = np.fft.rfftfreq(n,dT)
    
    return P2

In [ ]:
def linear_fit(k,Y,low_b,up_b):

    par = np.polyfit(np.log10(k[(k>low_b) & (k<up_b)]), np.log10(Y[(k>low_b) & (k<up_b)]), 1)
    slope = par[0]
    intercept = par[1]
    
    return slope, intercept   

In [ ]:
def psd_area(k,Y,b):
    
    x = k[k<b]
    y = Y[k<b]
    area = np.trapz(y,x)
    return area

# Calculate PSDs from telemetry files

Define spatial frequencies (34 bins)

In [ ]:
k = radialProfile(kr)

Store spatial PSD values in DataFrame for later analysis

In [ ]:
df_sp_psd = pd.DataFrame(columns = name_list)

i=0
for file in fname_list:
    print(file)
    print(name_list[i])
    y_2D = process_phase(file)
    y_psd = sp_power_spec(y_2D)
    df_sp_psd[name_list[i]] = y_psd
    i=i+1

In [ ]:
df_sp_psd.to_csv(save_path+'sp_psd'+'_'+dstr+'.txt')

Store temporal PSD values in DataFrame for later analysis.
Key = 'telemetry date'
columns = 'kt' & 't_psd' (Note varying timeseries lengths) 

In [ ]:
df_t_psd = pd.DataFrame()

i=0
for file in fname_list[0:2]:
    y_2D = process_phase(file)
    y_1D = np.mean(y_2D,axis=(1,2))
    y_psd = temp_power_spec(y_1D,.001)
    y_smoothed = 10**signal.savgol_filter(np.log10(y_psd), 101, 5)
    #df_t_psd[name_list[i],'dts'] = pd.to_datetime(name_list[i],format='%Y.%m.%d_%H.%M.%S',utc=True)
    new_df = pd.DataFrame({name_list[i]:y_smoothed})
    df_t_psd = pd.concat([df_t_psd,new_df],axis=1)
    print(name_list[i])
    i=i+1

In [ ]:
df_t_psd.to_csv(save_path+'t_psd'+'_'+dstr+'.txt')

# Grouping of samples 

read in spatial or temporal psd samples

In [ ]:
df_sp_psd = pd.read_csv(rootdir+'c_Eri_sp_psd_20190318.txt',index_col=0)
df_t_psd = pd.read_csv(rootdir+'c_Eri_t_psd_20190318.txt',index_col=0)

Put dates into night groups

In [ ]:
df = pd.DataFrame({'fname_list':fname_list,'name_list':name_list})
df['dts'] = pd.to_datetime(df['name_list'],format='%Y.%m.%d_%H.%M.%S',utc=True)
df['delta_t'] = df['dts']-df['dts'].shift(1)
df.loc[0,'delta_t'] = pd.Timedelta(0)

def bin_f(row):
    if row['delta_t'] > pd.Timedelta('20H'):
        return row['dts'].date()
    
df['bin']=df.apply(bin_f,axis = 1)
df.loc[0:1,'bin']=df.loc[0,'dts'].date()
df['bin']=df['bin'].fillna(method='ffill')
g_df = df.groupby('bin')

Create a dataframe with the average psd measured in each night window. Column names contain date at the start of the night 

In [ ]:
df_avg_sp_psd = pd.DataFrame()

for name,group in g_df:
    df_avg_sp_psd.loc[:,str(name)] = df_sp_psd.loc[:,group['name_list'].tolist()].mean(axis=1)

In [ ]:
df_avg_sp_psd.to_csv(save_path+'avg_sp_psd_'+dstr+'.txt')

In [ ]:
df_avg_t_psd = pd.DataFrame()

for name,group in g_df:
    df_avg_t_psd.loc[:,str(name)] = df_t_psd.loc[:,group['name_list'].tolist()].mean(axis=1)

In [ ]:
df_avg_t_psd.to_csv(save_path+'avg_t_psd_'+dstr+'.txt')

# Read in avg psd dataframes

In [ ]:
df_avg_sp_psd = pd.read_csv(rootdir+'c_Eri_avg_sp_psd_20190318.txt',index_col=0)
df_avg_t_psd = pd.read_csv(rootdir+'c_Eri_avg_t_psd_20190318.txt',index_col=0)

# Fit power law to avg PSDs

In [ ]:
%matplotlib notebook
df_psd_results = pd.DataFrame(columns=['sp_psd_slope','sp_psd_int','sp_psd_area','t_psd_slope','t_psd_int','t_psd_area'], index = df_avg_sp_psd.columns)
df_psd_results

for i in df_avg_sp_psd.columns:
    y = df_avg_sp_psd[i]
    x = radialProfile(kr)
    df_psd_results.loc[i,['sp_psd_slope','sp_psd_int']] = linear_fit(x,y,.33,1)
    df_psd_results.loc[i,'sp_psd_area'] = psd_area(x,y,2.8)
    

In [ ]:
df_psd_results.sort_values(by ='sp_psd_int',ascending=False)

In [ ]:
for i in df_avg_t_psd.columns:
    y = df_avg_t_psd[i].dropna()
    x = fft.fftfreq(len(y),.001)
    df_psd_results.loc[i,['t_psd_slope','t_psd_int']] = linear_fit(x,y,2,30)
    df_psd_results.loc[i,'t_psd_area'] = psd_area(x,y,400)

In [ ]:
df_psd_results.index

# Plot PSDs

Spatial PSD plot

In [ ]:
%matplotlib notebook

fit_x = np.linspace(0.33,1,20)

fig,ax = plt.subplots(1,1)

for key in df_psd_results.index:
    y = df_avg_sp_psd[key]
    m,yint = linear_fit(k,y,0.33,1)
    ax.loglog(k,y,label = key+ ' yint =' +str (yint.round(2)))
    #ax.loglog(fit_x,10**yint*fit_x**m)
    
ax.set_xlim(.1,3)
fig.legend(loc=1,fontsize=10)
    

In [ ]:
%matplotlib notebook

x = radialProfile(kr)
sp_bounds = (x>0.33) & (x<1)
fit_x = np.linspace(0.33,1,20)

y1 = df_avg_sp_psd['2016-09-18']
m1,int1 = linear_fit(x,y1,0.33,1)
print(m1,int1)

y2 = df_avg_sp_psd['2016-09-21']
m2,int2 = linear_fit(x,y2,0.33,1) 
print(m2,int2)

#plt.loglog(x,10**int2*x**-3.67,'silver',linestyle='-.')
plt.loglog(x,y1,'blue',lw=1,label = '2016-09-18 ; delT = .1')
plt.loglog(x,y2,'red',lw=1, label = '2016-09-21 ; delT = 1.1')

plt.loglog(fit_x,10**int1*fit_x**m1,'b|', marker=(2,0,-40),markersize=7)
plt.loglog(fit_x,10**int2*fit_x**m2,'r|',lw=1,marker=(2,0,-40),markersize=7)

plt.legend()
plt.grid(axis='both')
plt.xlim(.1,3)
#plt.ylim(3*10**-4,8)


Temporal PSD plot

In [ ]:
%matplotlib notebook

# spatial PSD calculations
sp_x = radialProfile(kr)
sp_fit_x = np.linspace(0.33,1,20)

sp_y1 = df_avg_sp_psd['2016-09-18']
sp_m1,sp_int1 = linear_fit(sp_x,sp_y1,0.33,1)
print(sp_m1,sp_int1)

sp_y2 = df_avg_sp_psd['2016-09-21']
sp_m2,sp_int2 = linear_fit(sp_x,sp_y2,0.33,1) 
print(sp_m2,sp_int2)

#sp_y3 = df_avg_sp_psd['2016-09-21']
#sp_m3,sp_int3 = linear_fit(sp_x,sp_y3,0.33,1) 
#print(sp_m3,sp_int3)

# temporal PSD calculations
t_y1 = df_avg_t_psd['2016-09-18']
t_x1 = fft.rfftfreq(len(t_y1),.001)
t_fit_x= np.linspace(3.0,30.0,2)
t_m1,t_int1 = linear_fit(t_x1,t_y1,3.0,30.0) 
print(t_m1,t_int1)

t_y2 = df_avg_t_psd['2016-09-21']
t_x2 = fft.rfftfreq(len(t_y2),.001)
t_m2,t_int2 = linear_fit(t_x2,t_y2,3.0,40.0) 
print(t_m2,t_int2)

#t_y3 = df_avg_t_psd['2016-09-21']
#t_x3 = fft.rfftfreq(len(t_y3),.001)
#t_m3,t_int3 = linear_fit(t_x3,t_y3,3.0,40.0) 
#print(t_m3,t_int3)

#Plotting in figure 
fig, (ax1,ax2) = plt.subplots(1,2,figsize=(18,6))

ax1.loglog(sp_x,sp_y1,'k:',lw=2,alpha = 1,label ='09/18/2016 PSD')
ax1.loglog(sp_x,sp_y3,'k--',lw=2,alpha = 1,label ='09/21/2016 PSD')
ax1.loglog(sp_x,sp_y2,'k',lw=2,alpha = 1,label ='12/19/2015 PSD')
ax1.loglog(sp_fit_x,10**sp_int1*sp_fit_x**sp_m1,'dodgerblue',lw=4,label ='09/18/2016 fit')
#ax1.loglog(sp_fit_x,10**sp_int3*sp_fit_x**sp_m3,'orange',lw=2, label ='09/21/2016 fit')
ax1.loglog(sp_fit_x,10**sp_int2*sp_fit_x**sp_m2,'crimson',lw=4, label ='12/19/2015 fit')

ax1.legend()
ax1.tick_params(axis='both',which = 'both',direction='in', width=1,top=True,right=True,length = 10, pad =15)
ax1.set_ylabel(r'PSD [$\frac{\mu m^2}{m^{-1}}$]')
ax1.set_xlabel(r'f [$m^{-1}$]')
ax1.set_xlim(.1,3)

ax1.annotate('slope = ' + str(sp_m1.round(2)),[.18,1.1*10**-2],fontsize =15,color = 'dodgerblue') # fit for 9/18/2018
ax1.annotate('slope = ' + str(sp_m2.round(2)),[.9,3*10**-3],fontsize =15,color = 'crimson') # fit for 9/21/2018
.15,1.1*10**-2
ax2.loglog(t_x1,t_y1,'k:',lw=2,alpha = 1,label ='09/18/2016 PSD')
ax2.loglog(t_x2,t_y2,'k',lw=2,alpha = 1,label ='09/21/2016 PSD')
ax2.loglog(t_fit_x,10**t_int1*t_fit_x**t_m1,'dodgerblue',lw=4,label ='09/18/2016 fit')
ax2.loglog(t_fit_x,10**t_int2*t_fit_x**t_m2,'crimson',lw=4, label ='09/21/2016 fit')

ax2.annotate('slope = ' + str(t_m1.round(2)),[3,1.1*10**-8],fontsize =15,color = 'dodgerblue')
ax2.annotate('slope = ' + str(t_m2.round(2)),[11,5*10**-7],fontsize =15,color = 'crimson')
15,5*10**-7
ax2.legend()
ax2.tick_params(axis='both',which = 'both',direction='in', width=1,top=True,right=True,length = 10, pad =15)
ax2.set_ylabel(r'PSD [$\frac{\mu m^2}{Hz}$]')
ax2.set_xlabel(r'f [$Hz$]')
ax2.set_xlim(.1,400)
#ax2.set_ylim(10**-9,10**-4)

fig.tight_layout()
fig.subplots_adjust(wspace=0.3)

plt.savefig(save_path+'c_Eri_PSD_'+dstr+'.eps',dpi = 300)

Plotting of PSDs

# Quick check of standard deviation

In [ ]:
phase = import_fits(fname_list[100])
m1 = gpipsfs.GeminiPrimary().sample(npix=48)

In [ ]:
std_phase = np.std(phase*m1,axis=0)

In [ ]:
%matplotlib notebook

plt.imshow(std_phase)